1. Напишите на Python программу, которая записывает произвольную html страницу, производит поиск содержащихся на ней персональных данных, анонимизирует содержимое и генерирует анонимизированный html файл для просмотра браузером.

2. Протестируйте вашу программу на 10 произвольных страницах. Вычислите, сколько процентов словосочетаний, являющихся персональными данными оставляет на странице ваша программа.

3. Подготовьте отчет с описанием программы , теста и демонстрацию.

In [100]:
from presidio_analyzer import AnalyzerEngine
from presidio_analyzer.nlp_engine import NlpEngineProvider
from presidio_anonymizer import AnonymizerEngine
from presidio_anonymizer.entities.engine import RecognizerResult, OperatorConfig


In [101]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver

**using spacy model ru_core_news_lg-3.2.0**

In [102]:
configuration = {
    "nlp_engine_name": "spacy",
    "models": [{"lang_code": "ru", "model_name": "ru_core_news_lg"}]
}

# Create NLP engine based on configuration
provider = NlpEngineProvider(nlp_configuration=configuration)
nlp_engine_with_russian = provider.create_engine()

# Pass the created NLP engine and supported_languages to the AnalyzerEngine
analyzer = AnalyzerEngine(
    nlp_engine=nlp_engine_with_russian, 
    supported_languages=["ru"]
)


def analyze_text(analyzer, text):
    results_russian = analyzer.analyze(text=text,
                                       score_threshold=0.6,
                                       language="ru")
    return(results_russian)

anonymizer_engine = AnonymizerEngine()
def anonymize_text(anonymizer, text, analyze_text):
    result = engine.anonymize(text=text, 
                              analyzer_results=analyze_text)
    return result


In [197]:
def get_urls_by_category(start_url, category_name, urls_number = 10):
    chrome_options = webdriver.ChromeOptions()
    chrome_options.add_argument('--headless')
    chrome_options.add_argument('--no-sandbox')
    chrome_options.add_argument('--disable-dev-shm-usage')

    driver = webdriver.Chrome('C:/Users/psmolnik/Downloads/chromedriver_win32/chromedriver.exe', options=chrome_options)
    driver.switch_to.window(driver.window_handles[0])
    
    urls = []
    driver.get(start_url + category_name)
    contents = driver.find_elements_by_class_name("list-item__content")
    for content in contents:
        urls.append(content.find_element_by_css_selector('a').get_attribute('href'))
    next_page = driver.find_element_by_class_name("list-more")
    next_page_url = start_url + next_page.get_attribute("data-url")
    urls_count = len(urls)

    while(urls_count < urls_number):
        driver.get(next_page_url)
        contents = driver.find_elements_by_class_name("list-item__content")
        for content in contents:
            urls.append(content.find_element_by_css_selector('a').get_attribute('href'))
        urls_count += len(contents)
        next_page = driver.find_element_by_class_name("list-items-loaded")
        next_page_url = start_url + next_page.get_attribute("data-next-url")

    driver.close()
    return urls[:10]

In [198]:
start_url = "https://ria.ru/"
categories = ["politics"]
urls = []
for category in categories:
    urls = get_urls_by_category(start_url, category)

C:\Users\psmolnik\AppData\Local\Temp/ipykernel_21184/1559786977.py:7: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('C:/Users/psmolnik/Downloads/chromedriver_win32/chromedriver.exe', options=chrome_options)
C:\Users\psmolnik\AppData\Local\Temp/ipykernel_21184/1559786977.py:12: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  contents = driver.find_elements_by_class_name("list-item__content")
C:\Users\psmolnik\AppData\Local\Temp/ipykernel_21184/1559786977.py:15: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  next_page = driver.find_element_by_class_name("list-more")


In [199]:
urls

['https://ria.ru/20211109/mid-1758254041.html',
 'https://ria.ru/20211109/posol-1758250560.html',
 'https://ria.ru/20211109/gosduma-1758230843.html',
 'https://ria.ru/20211109/vaktsinatsiya-1758212561.html',
 'https://ria.ru/20211109/putin-1758207360.html',
 'https://ria.ru/20211109/vstrecha-1758206975.html',
 'https://ria.ru/20211109/mosgorizbirkom-1758202918.html',
 'https://ria.ru/20211109/krym-1758176325.html',
 'https://radiosputnik.ria.ru/20211108/stratplanirovanie-1758132941.html',
 'https://radiosputnik.ria.ru/20211108/nikaragua-1758129984.html']

In [192]:
sourse_pwd = "./data_seminar2/source_pages/"
anonymize_pwd = "./data_seminar2/anonymized_pages/"

In [200]:
headers = requests.utils.default_headers()
headers.update({ 'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) Gecko/20100101 Safari/537.36'})

for i, url in enumerate(urls):
    print("processing", url)
    try:
        page = requests.get(url).text

        soup = BeautifulSoup(page)

        for link in soup.find_all('a', href=True):
            link.replace_with(link.text)

        title = soup.find("meta",  {"name":"analytics:title"})["content"]
        analyze_title = analyze_text(analyzer, title)
        result_title = anonymize_text(anonymizer_engine, title, analyze_title)
        locations = soup.findAll(text=title)
        for loc in locations:
            loc.replace_with(result_title.text)

        second_title = soup.find("h1", attrs={'class':'article__second-title'})
        analyze_second_title = analyze_text(analyzer, second_title.text)
        result_second_title = anonymize_text(anonymizer_engine, second_title.text, analyze_second_title)
        second_title.replace_with(result_second_title.text)

        text_body = soup.findAll("div", attrs={'class':'article__text'})
        for text in text_body:
            analyze_text_body = analyze_text(analyzer, text.text)
            result_text_body = anonymize_text(anonymizer_engine, text.text, analyze_text_body)
            text.replace_with(result_text_body.text)   

        tags = soup.find("meta",  {"name":"analytics:tags"})["content"].split(',')
        for tag in tags:
            tag = tag.strip()
            analyze_tag = analyze_text(analyzer, tag)
            result_tag = anonymize_text(anonymizer_engine, tag, analyze_tag)

            locations = soup.findAll(text=tag)
            for loc in locations:
                loc.replace_with(result_tag.text)
        
        f = open(sourse_pwd + "page" + str(i) + ".html", "w+", encoding='utf-8')
        f.write(page)
        f.close()
        
        f = open(anonymize_pwd + "page" + str(i) + ".html", "w+", encoding='utf-8')
        f.write(soup.prettify())
        f.close()
    except:
        print("not successfully", url)
        pass

processing https://ria.ru/20211109/mid-1758254041.html
processing https://ria.ru/20211109/posol-1758250560.html
processing https://ria.ru/20211109/gosduma-1758230843.html
processing https://ria.ru/20211109/vaktsinatsiya-1758212561.html
processing https://ria.ru/20211109/putin-1758207360.html
processing https://ria.ru/20211109/vstrecha-1758206975.html
processing https://ria.ru/20211109/mosgorizbirkom-1758202918.html
processing https://ria.ru/20211109/krym-1758176325.html
processing https://radiosputnik.ria.ru/20211108/stratplanirovanie-1758132941.html
processing https://radiosputnik.ria.ru/20211108/nikaragua-1758129984.html
not successfully https://radiosputnik.ria.ru/20211108/nikaragua-1758129984.html
